# Tutorial: Generating Structured Output with OpenAIChatGenerator

- **Level**: Intermediate
- **Time to complete**: 15 minutes
- **Prerequisites**: You must have an API key from an active OpenAI account as this tutorial is using the gpt-4o-mini model by OpenAI.
- **Components Used**: `ChatPromptBuilder`, `OpenAIChatGenerator`, `OutputValidator` (Custom component)
- **Goal**: Learn how to generate structured outputs with `OpenAIChatGenerator` using Pydantic model or Json schema. Optionally, use a custom `OutputValidator` to confirm the output matches the provided Pydantic model.

## Overview
This tutorial shows how to produce structured outputs by either providing [Pydantic](https://github.com/pydantic/pydantic) model or JSON schema to `OpenAIChatGenerator`. We’ll use `OutputValidator` to verify the generated output against the schema.

Note: Only latest model starting with `gpt-4o-mini` can be used for this feature.


## Preparing the Colab Environment

Enable the debug mode of logging:

In [1]:
import logging

logging.basicConfig()
logging.getLogger("canals.pipeline.pipeline").setLevel(logging.DEBUG)

## Installing Dependencies
Install Haystack and [colorama](https://pypi.org/project/colorama/) with pip:

In [ ]:
%%bash

pip install "haystack-ai>=2.18.0"
pip install colorama

## Structured output using Pydantic Model

First, we'll see how to pass Pydantic model to `OpenAIChatGenerator`. For this purpose, we define two [Pydantic models](https://docs.pydantic.dev/1.10/usage/models/), `City` and `CitiesData`. These models specify the fields and types that represent the data structure we want.

In [3]:
from typing import List
from pydantic import BaseModel


class City(BaseModel):
    name: str
    country: str
    population: int


class CitiesData(BaseModel):
    cities: List[City]

> You can change these models according to the format you wish to extract from the text.

## Creating a Custom Component: OutputValidator

`OutputValidator` is a custom component that validates if the JSON object the LLM generates complies with the provided [Pydantic model](https://docs.pydantic.dev/1.10/usage/models/). If it doesn't, OutputValidator returns an error message.

For more details about custom components, see [Creating Custom Components](https://docs.haystack.deepset.ai/docs/custom-components).

In [5]:
import json
import pydantic
from pydantic import ValidationError
from typing import Optional, List
from colorama import Fore
from haystack import component
from haystack.dataclasses import ChatMessage


# Define the component input parameters
@component
class OutputValidator:
    def __init__(self, pydantic_model: pydantic.BaseModel):
        self.pydantic_model = pydantic_model
        self.iteration_counter = 0

    # Define the component output
    @component.output_types(valid_replies=List[str], invalid_replies=Optional[List[str]], error_message=Optional[str])
    def run(self, replies: List[ChatMessage]):

        ## Try to parse the LLM's reply ##
        # If the LLM's reply is a valid object, return `"valid_replies"`
        try:
            output_dict = json.loads(replies[0].text)
            self.pydantic_model.model_validate(output_dict)
            print(
                Fore.GREEN
                + f"Valid JSON from LLM"
            )
            return {"valid_replies": replies}

        # If the LLM's reply is corrupted or not valid, return "invalid_replies" and the "error_message"
        except (ValueError, ValidationError) as e:
            print(
                Fore.RED
                + f"Output from LLM:\n {replies[0]} \n"
                f"Error from OutputValidator: {e}"
            )
            return {"invalid_replies": replies, "error_message": str(e)}

Then, create an OutputValidator instance with `CitiesData` that you have created before.

In [6]:
output_validator = OutputValidator(pydantic_model=CitiesData)

## Create a Prompt for LLM

Use `ChatPromptBuilder` in the pipeline to pass the user’s message to `OpenAIChatGenerator`.
For information about Jinja2 template and ChatPromptBuilder, see [ChatPromptBuilder](https://docs.haystack.deepset.ai/docs/chatpromptbuilder).

In [7]:
from haystack.components.builders import ChatPromptBuilder

prompt_template = [ChatMessage.from_user("User Input: {{passage}}")]
prompt_builder = ChatPromptBuilder(template=prompt_template)

## Initalizing the ChatGenerator to produce structured outputs

[OpenAIChatGenerator](https://docs.haystack.deepset.ai/docs/openaichatgenerator) generates
text using OpenAI's `gpt-4o-mini` model by default. We pass our Pydantic model to `response_format` paramter in generation_kwargs .

We also need to set the `OPENAI_API_KEY` variable.

Note: You can also set the `response_format` param in the `run` method of chat generator.

In [8]:
import os
from getpass import getpass

from haystack.components.generators.chat import OpenAIChatGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
chat_generator = OpenAIChatGenerator(generation_kwargs={"response_format": CitiesData})

## Building the Pipeline

Add all components to your pipeline and connect them.

In [9]:
from haystack import Pipeline

pipeline = Pipeline(max_runs_per_component=5)

# Add components to your pipeline
pipeline.add_component(instance=prompt_builder, name="prompt_builder")
pipeline.add_component(instance=chat_generator, name="llm")
pipeline.add_component(instance=output_validator, name="output_validator")

# Now, connect the components to each other
pipeline.connect("prompt_builder.prompt", "llm.messages")
pipeline.connect("llm.replies", "output_validator")


🚅 Components
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
  - output_validator: OutputValidator
🛤️ Connections
  - prompt_builder.prompt -> llm.messages (list[ChatMessage])
  - llm.replies -> output_validator.replies (list[ChatMessage])

### Visualize the Pipeline

Draw the pipeline with the [`draw()`](https://docs.haystack.deepset.ai/docs/drawing-pipeline-graphs) method to confirm the connections are correct. You can find the diagram in the Files section of this Colab.

In [10]:
# pipeline.draw("auto-correct-pipeline.png")

## Testing the Pipeline

Run the pipeline with an example passage that you want to convert into a JSON format and the `json_schema` you have created for `CitiesData`. For the given example passage, the generated JSON object should be like:
```json
{
  "cities": [
    {
      "name": "Berlin",
      "country": "Germany",
      "population": 3850809
    },
    {
      "name": "Paris",
      "country": "France",
      "population": 2161000
    },
    {
      "name": "Lisbon",
      "country": "Portugal",
      "population": 504718
    }
  ]
}
```
The output of the LLM should be compliant with the `json_schema`. If the LLM doesn't generate the correct JSON object, it will loop back and try again.

In [11]:
passage = "Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718."
result = pipeline.run({"prompt_builder": {"passage": passage}})

Valid JSON from LLM


### Print the Correct JSON
If you didn't get any error, you can now print the corrected JSON.

In [12]:
valid_reply = result["output_validator"]["valid_replies"][0].text
valid_json = json.loads(valid_reply)
print(valid_json)

{'cities': [{'name': 'Berlin', 'country': 'Germany', 'population': 3850809}, {'name': 'Paris', 'country': 'France', 'population': 2161000}, {'name': 'Lisbon', 'country': 'Portugal', 'population': 504718}]}


## Structured output using Json schema

Now, we’ll create a json schema of the `CitiesData` model and pass it to `OpenAIChatGenerator`. OpenAI expects schemas in a specific format, so the schema generated with `model_json_schema()` cannot be used directly.

For details on how to create schemas for OpenAI, see the [OpenAI Structured Outputs guide](https://platform.openai.com/docs/guides/structured-outputs#supported-schemas).

In [13]:
cities_data_schema={
        "type": "json_schema",
        "json_schema": {
            "name": "CitiesData",
            "schema": {
                "type": "object",
                "properties": {
                    "cities": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "name": { "type": "string" },
                                "country": { "type": "string" },
                                "population": { "type": "integer" }
                            },
                            "required": ["name", "country", "population"],
                            "additionalProperties": False
                        }
                    }
                },
                "required": ["cities"],
                "additionalProperties": False
            },
            "strict": True
        }
    }

Pass this json schema to the `response_format` parameter in chat generator. We run the generator indivdually to see the output.

In [14]:
chat_generator = OpenAIChatGenerator(generation_kwargs={"response_format": cities_data_schema})

messages = [ChatMessage.from_user("Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718.")]

result = chat_generator.run(messages=messages)

print(result["replies"][0].text)

{"cities":[{"name":"Berlin","country":"Germany","population":3850809},{"name":"Paris","country":"France","population":2161000},{"name":"Lisbon","country":"Portugal","population":504718}]}


## What's next

🎉 Congratulations! You've learnt how to easily produce structured ouputs with `OpenAIChatGenerator` using Pydantic models and Json schema.

To stay up to date on the latest Haystack developments, you can [subscribe to our newsletter](https://landing.deepset.ai/haystack-community-updates) and [join Haystack discord community](https://discord.gg/haystack).

Thanks for reading!